<a href="https://colab.research.google.com/github/sk-aili/market-basket-analysis/blob/main/Market_Basket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender


## Importing Libraries

In [ ]:
! pip install squarify gapminder --quiet

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import networkx as nx
import warnings
import matplotlib as mpl
import gapminder as gapminder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn.preprocessing import LabelEncoder
from mlxtend.frequent_patterns import fpgrowth


%matplotlib inline

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Data Understanding

In [ ]:
customer = pd.read_csv("/content/customer.csv")
print("Customer data")
customer.head()
product=pd.read_csv("/content/product.csv")
print("Product data")
product.head()
product_class=pd.read_csv("/content/product_class.csv")
print("Product Class data")
product_class.head()
region=pd.read_csv("/content/region.csv")
print("Region data")
region.head()
df=pd.read_csv("/content/sales.csv")
print("Sales data")
df.head()
store=pd.read_csv("/content/store.csv")
print("Store data")
store.head()
time_by_day=pd.read_csv("/content/time_by_day.csv")
print("Time By Day data")
time_by_day.head()
time_by_day.tail()



In [ ]:
df=df.merge(customer,on='customer_id')
#df.head()
df=df.merge(product,on='product_id')
#df.head()
df=df.merge(product_class,on='product_class_id')
#df.head()
df=df.merge(store,on='store_id')
#df.head()
df=df.merge(region,on='region_id')
#df.head()
df=df.merge(time_by_day,on='time_id')
df.head()
df.shape


In [ ]:
df.describe()
df.describe(include=['O'])
df.dtypes

In [ ]:
temp = pd.DataFrame(index=df.columns)
temp['data_type']=df.dtypes
temp['null_count']=df.isnull().sum()
temp['unique_count']=df.nunique()
temp

### Univariate Analysis

In [ ]:
fig=plt.figure(figsize=(8,6))
df['product_class_id'].plot.hist(bins = 50)
plt.title("Frequency plot of Department ID")
plt.xlabel('department id')
plt.ylabel('Frequency')

In [ ]:
fig=plt.figure(figsize=(8,6))
df['product_id'].plot.hist(bins = 100)
plt.title("Frequency plot of Product ID")
plt.xlabel('product id')
plt.ylabel('Frequency')

In [ ]:
fig=plt.figure(figsize=(6,4))
df['unit_sales'].plot.hist(bins = 25)
plt.title("Frequency plot of Unit Sales")
plt.xlabel('Unit Sales')
plt.ylabel("Fr")

In [ ]:
## When does people order?
fig=plt.figure(figsize=(8, 6))
df['the_year'].plot.hist(bins=10)
plt.title("Frequency of orders by Year")
plt.xlabel('year')
plt.xticks([1996,1997,1998,1999])
plt.ylabel("Frequency")
plt.show()

In [ ]:
fig=plt.figure(figsize=(8,6))
df['the_month'].plot.hist(bins=50)
plt.title('Frequency of orders by month')
plt.xlabel('Month')
plt.ylabel('Frequency')
plt.show()

In [ ]:
fig=plt.figure(figsize=(8,6))
df['day_of_month'].plot.hist(bins=75)
plt.title("Frequency of orders by day of a month")
plt.xlabel('Day of month')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df_day_freq=df['the_day'].value_counts()
fig=plt.figure(figsize=(8, 6))
df_day_freq.plot.bar()
plt.title("Frequency of orders by day of a Week")
plt.xlabel('Day of Week')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df['products'] = 'Products'
products = df.truncate(before = 605, after = 615)
products = nx.from_pandas_edgelist(products, source = 'products', target = 'product_name', edge_attr = True)
plt.rcParams['figure.figsize']=(12, 12)
pos=nx.spring_layout(products)
nx.draw_networkx_nodes(products,pos,node_size=10000,node_color='grey')
nx.draw_networkx_edges(products, pos, width = 1, alpha = 0.6, edge_color = 'black')
nx.draw_networkx_labels(products, pos, font_size = 10)
plt.axis('off')
plt.grid()
plt.title('Top 10 First Choices in Products')
plt.show()

In [ ]:
## What are the most ordered products?
# df['product_name'].value_counts()
df['product_name'].value_counts().head(10)/len(df)*100

In [ ]:
df_freq_products=df['product_name'].value_counts().iloc[:20]
fig=plt.figure(figsize=(8, 5))
df_freq_products.plot.bar()
plt.title("Barplot of Top 20 Bought Products")
plt.xlabel("Product Name")
plt.ylabel("Frequency")

In [ ]:
df['departments'] = 'Departments'
departments = df.truncate(before = 150, after = 195)
departments = nx.from_pandas_edgelist(departments, source = 'departments', target = 'department', edge_attr = True)
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize']=(12,12)
pos=nx.spring_layout(departments)
nx.draw_networkx_nodes(departments,pos,node_size=10000,node_color='orange')
nx.draw_networkx_edges(departments, pos, width = 1, alpha = 0.6, edge_color = 'black')
nx.draw_networkx_labels(departments, pos, font_size = 10)
plt.axis('off')
plt.grid()
plt.title('Top 10 First Choices in Departments', fontsize = 11)
plt.show()

In [ ]:
## what are the most visited departments?
# df['department'].value_counts()
df['department'].value_counts().head(10)/len(df)*100

In [ ]:
fig=plt.figure(figsize=(10,5))
df['department'].value_counts().plot(kind='bar')
plt.title("Barplot Of The Most Visited Department")
plt.xlabel("Department")
plt.ylabel("Frequency")

In [ ]:
## what are the most visited aisle?
# df['subcategory'].value_counts()
df['subcategory'].value_counts().head(10)/len(df)*100

In [ ]:
df_subcategory_freq=df['subcategory'].value_counts().iloc[:50]
fig=plt.figure(figsize=(10,5))
df_subcategory_freq.plot.bar()
plt.title("Barplot Of The Most Visited Aisle")
plt.xlabel('Aisle')
plt.ylabel('Frequency')

**Array of Most Bought Product**

In [ ]:
y=df_freq_products.head(50).to_frame()
y.index

## Feature Engineering

In [ ]:
df.shape
df.drop_duplicates()
df.shape

### Bivariate Analysis

In [ ]:
data_user_orders_num=df.groupby('customer_id')['unit_sales'].count()
source_data = {}
for i in range(10):
    source_data[str(10*i)+'-'+str(10*(i+1))]=len([x for x in list(data_user_orders_num) if x>=i*10 and x<(i+1)*10])

font_size=10
fig_size=(8,6)
mpl.rcParams['font.size']=font_size
mpl.rcParams['figure.figsize']=fig_size
bar_width=0.3
x_axis = tuple(source_data.keys())
y_axis = tuple(source_data.values())
#assign color
plt.bar(x_axis, y_axis, color='orange')
# descrpitions for x-axis, y-axis
plt.xlabel('Unit sales')
plt.ylabel("No. of customers")
plt.title("Bar Plot Between Customers and their Products per Order")
plt.show()

In [ ]:
## Transaction ID - create transaction id which denotes a basket
df['transaction_id'] = df['customer_id'].astype(str) + df['time_id'].astype(str)
cols = [76,2,0,23]
order_products=df[df.columns[cols]]
## Average products bought by customers per order
data_user_products_num1=order_products.groupby('transaction_id')['product_id'].count()
data_user_products_num1=pd.DataFrame(data_user_products_num1)
data_user_products_num1['transaction_id']=list(data_user_products_num1.index)
data_user_products_num1.columns=['product_num','orderid']
data_user_products_num2=pd.merge(data_user_products_num1,df[['transaction_id','customer_id']],on='transaction_id',how='left')
data_user_products_num3=data_user_products_num2.groupby('customer_id')['product_num'].agg(['sum','count'])
data_user_products_num3['avg']=data_user_products_num3['sum']/data_user_products_num3['count']
data_user_products_num3.head()

In [ ]:
# Featured Products
cols = [1,36,38,24]
departments=df[df.columns[cols]]
temp=['department']
produce=df.loc[df['department'] == 'Produce']
top_produce=produce['product_name'].value_counts().iloc[:10]
snacks=df.loc[df['department']=='Snack Foods']
top_snacks=snacks['product_name'].value_counts().iloc[:10]
household=df.loc[df['department']=='Household']
top_household=household['product_name'].value_counts().iloc[:10]
frozen=df.loc[df['department']=='Frozen Foods']
top_frozen=frozen['product_name'].value_counts().iloc[:10]
baking=df.loc[df['department']=='Baking Goods']
top_baking=baking['product_name'].value_counts().iloc[:10]
canned=df.loc[df['department']=='Canned Foods']
top_canned=canned['product_name'].value_counts().iloc[:10]
dairy=(df.loc[df['department'] == 'Dairy'])
top_dairy=dairy['product_name'].value_counts().iloc[:10]
dairy=(df.loc[df['department'] == 'Health and Hygiene'])
top_dairy=dairy['product_name'].value_counts().iloc[:10]
dairy=(df.loc[df['department'] == 'Beverages'])
top_dairy=dairy['product_name'].value_counts().iloc[:10]
dairy=(df.loc[df['department'] == 'Deli'])
top_dairy=dairy['product_name'].value_counts().iloc[:10]
dairy=(df.loc[df['department'] == 'Alcoholic Beverages'])
top_dairy=dairy['product_name'].value_counts().iloc[:10]
dairy=(df.loc[df['department'] == 'Starchy Foods'])
top_dairy=dairy['product_name'].value_counts().iloc[:10]
dairy=(df.loc[df['department'] == 'Eggs'])
top_dairy=dairy['product_name'].value_counts().iloc[:10]

In [ ]:
## Top 100 most frequently purchased products
cols = [76,2,0,23,6,1]
product_name=df[df.columns[cols]]
productCountDf = product_name.groupby("product_id",as_index = False)['transaction_id'].count()
productCountDf = productCountDf.sort_values("transaction_id",ascending = False)
topProdFrame = productCountDf.iloc[0:100,:]
productId= topProdFrame.loc[:,["product_id"]]

In [ ]:
## Orders containting the the most frequently purchased products
MarketBasketdf = product_name[0:0]
for i in range(0,99):
    pId = productId.iloc[i]['product_id']
    stDf = product_name[product_name.product_id == pId ]
    MarketBasketdf = MarketBasketdf.append(stDf,ignore_index = False)

basket = MarketBasketdf.groupby(['transaction_id','product_name'])['unit_sales'].sum().unstack().reset_index().fillna(0).set_index('transaction_id')

**One Hot Encoding**

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1


In [ ]:
basket_sets = basket.applymap(encode_units)
dummy=basket_sets.head(10000)

**Apriori Algorithm & its Recommendations**

In [ ]:
## Frequent items with support 0.01% using Apriori Algorithm
frequent_itemsets = apriori(dummy, min_support=0.0001, use_colnames=True)

In [ ]:
apriori_rules = association_rules(frequent_itemsets, metric="lift", min_threshold= 1)
apriori_rules[ (apriori_rules['lift'] >= 50) & (apriori_rules['confidence'] >= 0.01) ]

In [ ]:
def recommendations_using_Apriori(item):
    recommend = []
    for i in range(0,2646):
        if item == apriori_rules.iloc[i,0]:
            recommend.append(apriori_rules.iloc[i,1])

    return recommend

In [ ]:
## 5 Recommendations with Better Chicken Noodle Soup
product_name = {'Better Chicken Noodle Soup'}
recommentations=recommendations_using_Apriori(product_name)
print(*recommentations[0:5], sep = "\n")

In [ ]:
## 10 Recommendations with Moms Potato Salad
product_name = {'Moms Potato Salad'}
recommentations=recommendations_using_Apriori(product_name)
print(*recommentations[0:10], sep = "\n")

In [ ]:
## 15 Recommendations with Carrington Ice Cream Sandwich
product_name = {'Carrington Ice Cream Sandwich'}
recommentations=recommendations_using_Apriori(product_name)
print(*recommentations[0:15], sep = "\n")

**Fpgrowth Algorithm & its Recommendations**

In [ ]:
## Frequent Items with support 0.001% using Fpgrowth Algorithm
freq_items=fpgrowth(dummy,min_support=.0001,use_colnames=True)
## Association Rules using Fpgrowth Algorithm
fpgrowth_rules=association_rules(freq_items,metric="lift",min_threshold=1)

In [ ]:
def recommendations_using_Fpgrowth(item):
    recommend = []
    for i in range(0,2646):
        if item == fpgrowth_rules.iloc[i,0]:
            recommend.append(fpgrowth_rules.iloc[i,1])

    return recommend

In [ ]:
## 5 Recommendations with Better Chicken Noodle Soup
product_name = {'Better Chicken Noodle Soup'}
recommentations=recommendations_using_Fpgrowth(product_name)
print(*recommentations[0:5], sep = "\n")

In [ ]:
## 10 Recommendations with Moms Potato Salad
product_name = {'Moms Potato Salad'}
recommentations=recommendations_using_Fpgrowth(product_name)
print(*recommentations[0:10], sep = "\n")

In [ ]:
## 15 Recommendations with Carrington Ice Cream Sandwich
product_name = {'Carrington Ice Cream Sandwich'}
recommentations=recommendations_using_Fpgrowth(product_name)
print(*recommentations[0:15], sep = "\n")

**Apriori vs fpgrowth Algorithm: Performance measure**

In [ ]:
## Calculating Run Time of Apriori Algorithm
import time
l=[0.01,0.02,0.03,0.04,0.05]
t=[]
for i in l:
    t1=time.time()
    apriori(dummy,min_support=i,use_colnames=True)
    t2=time.time()
    t.append((t2-t1)*1000)

In [ ]:
## Calculating Run Time of Fpgrowth Algorithm
l=[0.01,0.02,0.03,0.04,0.05]
f=[]
for i in l:
    t1=time.time()
    fpgrowth(dummy,min_support=i,use_colnames=True)
    t2=time.time()
    f.append((t2-t1)*1000)

**Graph of Run Time between Apriori and Fpgrowth Algorithm**

In [ ]:
sns.lineplot(x=l,y=f,label="fpgrowth")
sns.lineplot(x=l,y=t,label="apriori")
plt.xlabel("Min_support Threshold")
plt.ylabel("Run Time in ms")

**Thank You**